In [ ]:
# vEnv - Instalações
%pip install pymupdf pymupdf4llm langchain

# 1. Pretratamento dos textos

In [1]:
import os
import pickle # Para asalvar arquivos intermediárioa
import re
import pymupdf
from pymupdf4llm import to_markdown

from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_core.documents import Document

In [2]:
# Identificar arquivos de document
DOC_FOLDER = "../Documentos/"
planos = os.listdir(DOC_FOLDER)
print(planos)

['pje-27259ff0-Plano de gov psolrede 2024 final.pdf', 'pje-NOVO PLANO DE GOVERNO_TRE.pdf', 'pje-Plano de campanha.pdf', 'PlanodeGovernoCristinaGraeml.pdf', 'PLANODEGOVERNOCURITIBAPODEMAISv4compressed1.pdf', 'PLANODEGOVERNODUCCIGOURA.pdf', 'PlanodeGovernoLuizoGoulart.pdf', 'PlanodegovernoRequio33.pdf', 'Programa2024.pdf', 'ProgramaparaasEleicoes2024.pdf']


In [ ]:
# Abiri um arquivo PDF para primeira verificação do texto

file = planos[0]
plano = pymupdf.open(DOC_FOLDER + file)

print("Plano de governo:", plano)
print("num de páginas", plano.page_count)

for i in range(plano.page_count):
    print("-> Página", i)
    print(plano.get_page_text(i))

Análise de páginas a excluir (capa, imagens, sumario, etc)

0 -> excluir páginas 0, 1 e 71

1 -> excluir páginas 0-7, 11, 13, 37, 62-63, 76-77

2 -> excluir páginas 0-2, 6

3 -> excluir páginas 0, 3, 21, 29, 57-fim

4 -> excluir páginas 0, 1, 

5 -> excluir páginas 0, 1, 8

6 -> excluir páginas 0

7 -> excluir páginas 

8 -> excluir páginas 0, 1

9 -> excluir páginas

In [ ]:
# Extração dos textos

paginas = [
    list(range(2, plano.page_count -1)),
    list(range(8, 11)) + [12] + list(range(14, 37)) + list(range(37, 62)) + list(range(64, 76)),
    list(range(3, plano.page_count)),
    [1, 2] + list(range(4, 21)) + list(range(22, 29)) + list(range(29, 57)),
    list(range(2, plano.page_count)),
    list(range(2, 8)) + list(range(9, plano.page_count)),
    list(range(1, plano.page_count)),
    list(range(0, plano.page_count)),
    list(range(2, plano.page_count)),
    list(range(0, plano.page_count))
    ]

headers_to_split_on = [
    [("#", "Header_1")],
    [("#", "Header_1"), ("##", "Header_2")],
    [("#", "Header_1")],
    [("###", "Header_1")],
    [("#", "Header_1")],
    [("#", "Header_1")],
    [("#####", "Header_1"), ("######", "Header_2")],
    [("#", "Header_1")],
    [("#", "Header_1")],
    [("##", "Header_1")],
    ]

def replace_headers(text): 
    """Replaces headers in a text with Markdown H1 headers.

    Args:
        text (str): The input text.

    Returns:
        str: The modified text with replaced headers.
    """

    # para 0: pattern = r"(\d+)- ([A-Z\s\.\,\Ç\Ã\:\Ú\Ê]+)\n"
    # para 0: replacement = r"# \2\n"

    pattern = r"\d+$"

    return re.sub(pattern, "", text, flags=re.MULTILINE)

all_docs = []
for plano, lists_pag in zip(planos, paginas):
    plano_md = to_markdown(plano, lists_pag)

    # limpar caracteres indesejados
    lista = [['###### GOVERNO CHICO', ''],
            ['# APRESENTAÇÃO', '### INTRODUÇÂO'],
            ['-----', ''],
            ["```", ""],
            ['_',''],
            ['\n\n', '\n'],
            ['  ', ' ']]

    for l in lista:
    plano_md = plano_md.replace(l[0], l[1])
    plano_md2 = replace_headers(plano_md) # Incluir markdowns quando não há
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
    plano_md_splits = splitter.split_text(plano_md)

    plano_md_splits_lc = []
    for doc in plano_md_splits:
        doc.metadata['source']=file
        plano_md_splits_lc.append(Document(page_content=doc.page_content, metadata=doc.metadata))

    all_docs = all_docs + plano_md_splits_lc

In [ ]:
#Adicionar candidatos (esqueci)

candidatos_plano = [
    ['pje-27259ff0-Plano de gov psolrede 2024 final.pdf', 'FEDERAÇÃO PSOL - REDE', 'PSOL / REDE', 'ANDREA DO ROCIO CALDAS', 'LETICIA FARIA'],
    ['pje-NOVO PLANO DE GOVERNO_TRE.pdf', 'Curitiba Amor e Inovação', 'NOVO / PL / MDB / REPUBLICANOS / PSD / PODE / AVANTE / PRTB', 'EDUARDO PIMENTEL SLAVIERO', 'Paulo Martins'],
    ['pje-Plano de campanha.pdf','CURITIBA MELHOR PARA TODOS', 'PP / PRD', 'MARIA VICTORIA BORGHETTI BARROS', 'Walter Petruzziello'],
    ['PlanodeGovernoCristinaGraeml.pdf', 'PMB', 'NA', 'CRISTINA REIS GRAEML', 'Dr Jairo Filho'],
    ['PLANODEGOVERNOCURITIBAPODEMAISv4compressed1.pdf', 'CURITIBA PODE MAIS', 'UNIÃO / AGIR / DC', 'NEY LEPREVOST NETO', 'Rosângela Moro'],
    ['PLANODEGOVERNODUCCIGOURA.pdf', 'Curitiba + Social e Humana', 'PSB / Federação BRASIL DA ESPERANÇA - FE BRASIL(PT/PC do B/PV) / PDT', 'LUCIANO DUCCI', 'Goura'],
    ['PlanodeGovernoLuizoGoulart.pdf', 'SOLIDARIEDADE', 'NA', 'LUIZ GOULARTE ALVES', 'Tiago Chico'],
    ['PlanodegovernoRequio33.pdf', 'MOBILIZA', 'NA', 'ROBERTO REQUIÃO DE MELLO E SILVA', 'Marcelo Henrique'],
    ['Programa2024.pdf', 'PSTU', 'NA', 'SAMUEL DE MATTOS FIGUEIREDO', 'Leo Martinez'],
    ['ProgramaparaasEleicoes2024.pdf', 'PCO', 'NA', 'FELIPE GUSTAVO BOMBARDELLI', 'Paulo Costycha']
    ] 

for doc in all_docs:
    for chapa in candidatos_plano:
        if doc.metadata['source'] == chapa[0]:
            doc.metadata['partido'] = chapa[1]
            doc.metadata['candidato'] = chapa[3]
            doc.metadata['vice'] = chapa[4]
            doc.metadata['coalizao'] =  chapa[2]
            

In [ ]:
# Padronizar STD Header

dicionario = [
    ['INTRODUÇÃO', 'Introdução'],
    ['EDUCAÇÃO', 'Educação'],
    ['SAÚDE','Saúde'],
    ['MOBILIDADE E MORADIA: DIREITO A CIDADE','Habitação e Ocupação Urbana'],
    ['ASSITÊNCIA SOCIAL','Assistência e Ação Social'],
    ['MEIO AMBIENTE','Meio Ambiente e crise climática'],
    ['SEGURANÇA','Segurança Pública'],
    ['CIDADE, CIDADANIA E DIVERSIDADE','Políticas Afirmativas e Direitos Humanos'],
    ['CULTURA, ESPORTE E LASER','Cultura, Esporte, Lazer e Turismo'],
    ['ADMINISTRAÇÃO E ECONOMIA', 'Gestão e Transparência'],
    ['DIREITOS HUMANOS','Políticas Afirmativas e Direitos Humanos'],
    ['IGUALDADE ÉTNICO-RACIAL','Políticas Afirmativas e Direitos Humanos'],
    ['MULHER','Políticas Afirmativas e Direitos Humanos'],
    ['PESSOA COM DEFICIÊNCIA','Políticas Afirmativas e Direitos Humanos'],
    ['AÇÃO SOCIAL', 'Assistência e Ação Social'],
    ['HABITAÇÃO SOCIAL','Habitação e Ocupação Urbana'],
    ['SEGURANÇA PÚBLICA / DEFESA SOCIAL','Segurança Pública'],
    ['DEFESA CIVIL','Meio Ambiente e crise climática'],
    ['ESPORTE, LAZER E JUVENTUDE','Cultura, Esporte, Lazer e Turismo'],
    ['CULTURA','Cultura, Esporte, Lazer e Turismo'],
    ['SEGURANÇA ALIMENTAR E NUTRICIONAL','Segurança Alimentar'],
    ['GESTÃO DEMOCRÁTICA','Gestão e Transparência'],
    ['INFRAESTRUTURA URBANA','Urbanismo'],
    ['PROTEÇÃO ANIMAL','Zeladoria e Serviços'],
    ['ADMINISTRAÇÃO EFICIENTE E TRANSPARENTE','Gestão e Transparência'],
    ['VALORIZAÇÃO DOS SERVIDORES PÚBLICOS', 'Zeladoria e Serviços'],
    ['INSTITUTO DE ADMINISTRAÇÃO PÚBLICA','Gestão e Transparência'],
    ['PLANEJAMENTO URBANO','Urbanismo'],
    ['REGIONAL BAIRRO NOVO','Urbanismo'],
    ['REGIONAL BOA VISTA','Urbanismo'],
    ['REGIONAL BOQUEIRÃO','Urbanismo'],
    ['REGIONAL CAJURU','Urbanismo'],
    ['REGIONAL CIC','Urbanismo'],
    ['REGIONAL MATRIZ','Urbanismo'],
    ['REGIONAL PORTÃO','Urbanismo'],
    ['REGIONAL SANTA FELICIDADE','Urbanismo'],
    ['TRANSPORTE PÚBLICO INTEGRADO','Transporte Público e Mobilidade'],
    ['TURISMO','Cultura, Esporte, Lazer e Turismo'],
    ['DESENVOLVIMENTO ECONÔMICO E INOVAÇÃO','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['INTEGRAÇÃO METROPOLITANA','Transporte Público e Mobilidade'],
    ['MOBILIDADE URBANA','Transporte Público e Mobilidade'],
    ['INFRAESTRUTURA E DESENVOLVIMENTO URBANO','Urbanismo'],
    ['SEGURANÇA PÚBLICA','Segurança Pública'],
    ['MEIO AMBIENTE, SUSTENTABILIDADE E ZELADORIA','Meio Ambiente e crise climática'],
    ['DESENVOLVIMENTO ECONOMICO','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['ASSISTÊNCIA SOCIAL','Assistência e Ação Social'],
    ['CULTURA, ESPORTES E LAZER','Cultura, Esporte, Lazer e Turismo'],
    ['HABITAÇÃO E SEGURANÇA ALIMENTAR','Habitação e Ocupação Urbana'],
    ['POLÍTICAS PÚBLICAS PARA MULHERES','Políticas Afirmativas e Direitos Humanos'],
    ['CONCLUSÃO','Conclusão'],
    ['Introdução','Introdução'],
    ['TECNOLOGIA E INOVAÇÃO','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['DESENVOLVIMENTO ECONÔMICO','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['MEIO AMBIENTE E SUSTENTABILIDADE','Meio Ambiente e crise climática'],
    ['ESPORTE','Cultura, Esporte, Lazer e Turismo'],
    ['GESTÃO PÚBLICA E ADMINISTRATIVA','Gestão e Transparência'],
    ['TRANSPORTE E MOBILIDADE URBANA','Transporte Público e Mobilidade'],
    ['URBANISMO E PLANEJAMENTO URBANO','Urbanismo'],
    ['DESENVOLVIMENTO PESSOAL E QUALIFICAÇÃO DE SERVIDORES PÚBLICOS','Funcionalismo Público'],
    ['DESENVOLVIMENTO RURAL E AGRICULTURA URBANA','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['QUALIDADE DO ATENDIMENTO','Gestão e Transparência'],
    ['GESTÃO','Gestão e Transparência'],
    ['ARTICULAÇÃO','Gestão e Transparência'],
    ['PARTICIPAÇÃO E CONTROLE SOCIAL','Gestão e Transparência'],
    ['C.R.I.S.T.O.S. – CENTROS DE REABILITAÇÃO, INCLUSÃO, SAÚDE, TRABALHO E ORGANIZAÇÃO.','Assistência e Ação Social'],
    ['POPULAÇÃO EM SITUAÇÃO DE RUA.','Assistência e Ação Social'],
    ['PROGRAMA DE HABITAÇÃO.','Habitação e Ocupação Urbana'],
    ['322 FAVELAS EM CURITIBA.','Habitação e Ocupação Urbana'],
    ['PROMOÇÃO DOS DIREITOS DAS MULHERES.','Políticas Afirmativas e Direitos Humanos'],
    ['POLÍTICAS AFIRMATIVAS.','Políticas Afirmativas e Direitos Humanos'],
    ['TRANSPARÊNCIA, SERVIÇOS PÚBLICOS E FUNCIONALISMO PÚBLICO.','Gestão e Transparência'],
    ['SUSTENTABILIDADE','Meio Ambiente e crise climática'],
    ['EDUCAÇÃO.','Educação'],
    ['TRABALHO E EMPREENDEDORISMO.','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['TURISMO E CULTURA.','Cultura, Esporte, Lazer e Turismo'],
    ['SEGURANÇA PÚBLICA MUNICIPAL E POLÍTICA DE TRÂNSITO.','Segurança Pública'],
    ['TRANSPORTE PÚBLICO.','Transporte Público e Mobilidade'],
    ['CONCLUSÃO.','Conclusão'],
    ['Educação','Educação'],
    ['Saúde','Saúde'],
    ['Segurança Pública','Segurança Pública'],
    ['Ambiente e Prevenção ao Aquecimento Global','Meio Ambiente e crise climática'],
    ['Tratamento dos Resíduos da Atividade Humana na Cidade','Meio Ambiente e crise climática'],
    ['Transporte Público Urbano e Mobilidade','Transporte Público e Mobilidade'],
    ['Infraestrutura Urbana','Urbanismo'],
    ['Habitação de Interesse Social e Regularização Fundiária','Habitação e Ocupação Urbana'],
    ['Conservação da cidade','Zeladoria e Serviços'],
    ['Trabalho, Empreendedorismo e Renda','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['Propostas para o Turismo','Cultura, Esporte, Lazer e Turismo'],
    ['Propostas para o Artesanato','Cultura, Esporte, Lazer e Turismo'],
    ['Segurança Alimentar e Nutricional','Segurança Alimentar'],
    ['Rede Comercial de Abastecimento','Segurança Alimentar'],
    ['Assistência Social e Cidadania','Assistência e Ação Social'],
    ['Ampliar a Participação Popular','Gestão e Transparência'],
    ['Atividade Física, Esporte e Lazer','Cultura, Esporte, Lazer e Turismo'],
    ['Cultura','Cultura, Esporte, Lazer e Turismo'],
    ['Prefácio','Introdução'],
    ['Inclusão e assistência social','Assistência e Ação Social'],
    ['Mulher, idoso e juventude','Políticas Afirmativas e Direitos Humanos'],
    ['Mulher','Políticas Afirmativas e Direitos Humanos'],
    ['Idoso','Políticas Afirmativas e Direitos Humanos'],
    ['Juventude','Políticas Afirmativas e Direitos Humanos'],
    ['Esporte e lazer','Cultura, Esporte, Lazer e Turismo'],
    ['Abastecimento','Segurança Alimentar'],
    ['Moradia','Habitação e Ocupação Urbana'],
    ['Trabalho, renda e desenvolvimento','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['Turismo','Cultura, Esporte, Lazer e Turismo'],
    ['Segurança pública e cidadania','Segurança Pública'],
    ['Urbanismo','Urbanismo'],
    ['Transporte e trânsito','Transporte Público e Mobilidade'],
    ['Meio ambiente e sustentabilidade','Meio Ambiente e crise climática'],
    ['Gestão pública e servidores','Gestão e Transparência'],
    ['Segurança e Ordem Pública','Segurança Pública'],
    ['Assistência Social','Assistência e Ação Social'],
    ['Tecnologia, inovação e Inteligência artificial','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['Inclusão e Proteção para Mulheres','Políticas Afirmativas e Direitos Humanos'],
    ['Igualdade Racial','Políticas Afirmativas e Direitos Humanos'],
    ['Infância e Juventude','Políticas Afirmativas e Direitos Humanos'],
    ['Idosos','Políticas Afirmativas e Direitos Humanos'],
    ['Esporte, lazer e turismo','Cultura, Esporte, Lazer e Turismo'],
    ['Mobilidade e infraestrutura','Transporte Público e Mobilidade'],
    ['Habitação','Habitação e Ocupação Urbana'],
    ['Meio ambiente','Meio Ambiente e crise climática'],
    ['Gestão pública e desenvolvimento econômico','Gestão e Transparência'],
    ['GESTÃO PÚBLICA','Gestão e Transparência'],
    ['TRANSPORTE PÚBLICO','Transporte Público e Mobilidade'],
    ['OCUPAÇÃO URBANA','Habitação e Ocupação Urbana'],
    ['TRABALHO','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['INTRODUÇÃ0','Introdução'],
    ['Dívida pública','Gestão e Transparência'],
    ['Transporte','Transporte Público e Mobilidade'],
    ['Salário mínimo para os funcionários municipais','Funcionalismo Público'],
    ['Calçamento e esgoto, água e luz','Urbanismo'],
    ['Plano de Emprego','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['Formação de conselhos populares deliberativos','Gestão e Transparência'],
    ['Camelôs e ambulantes','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['Diminuição do IPTU','Legislações Específicas / Outros'],
    ['Radares e a indústria da multa','Legislações Específicas / Outros'],
    ['Plano de Emprego','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['Crédito','Desenvolvimento Econômico, Emprego, Renda e Tecnologia'],
    ['Alimentos','Segurança Alimentar'],
    ['Praças e clubes','Cultura, Esporte, Lazer e Turismo'],
    ['Cancelamento das leis absurdas','Legislações Específicas / Outros']
    ]

for doc in all_docs:
    if 'Header_2' in doc.metadata:
        for d in dicionario:
            if d[0] == doc.metadata['Header_2']:
                doc.metadata['STD_Header'] = d[1]
    else:
        for d in dicionario:
            if d[0] == doc.metadata['Header_1']:
                doc.metadata['STD_Header'] = d[1] 

In [ ]:
# adicionar sequencial por plano e ID para documentos
ref_source = "nada"
seq = 0
for doc in all_docs:
    if doc.metadata['source'] == ref_source:
        seq = seq + 1
        doc.metadata['sequence'] = seq
    else:
        seq = 0
        ref_source = doc.metadata['source']
        doc.metadata['sequence'] = seq
    doc.id = doc.metadata['candidato'][0:5] + f"{seq:02d}"


In [ ]:
# SALVAR

with open('STD_documents.pkl', 'wb') as f:
    pickle.dump(all_docs, f)